# Data Collection 

In [11]:
import numpy as np
import pandas as pd 

# import tensorflow as tf
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense

# import re
# import string

# #!pip install langdetect
# from langdetect import detect

# import time
# # !pip install deep-translator
# from deep_translator import GoogleTranslator

# from nltk.tokenize import word_tokenize

In [15]:
colected_files = [
    '/kaggle/input/roman-text-data/collected_data/Daraz Labelled Review Dataset with Sentiments and Features.xlsx',
    '/kaggle/input/roman-text-data/collected_data/New.csv',
    '/kaggle/input/roman-text-data/collected_data/RU-EN-Emotion Dataset.xlsx',
    '/kaggle/input/roman-text-data/collected_data/Roman Urdu DataSet.csv',
    '/kaggle/input/roman-text-data/collected_data/Roman Urdu DataSet_first.csv',
    '/kaggle/input/roman-text-data/collected_data/Roman Urdu Tagged Dataset.csv',
    '/kaggle/input/roman-text-data/collected_data/Roman Urdu Toxic Dataset.xlsx',
    '/kaggle/input/roman-text-data/collected_data/RomanUrduHateSpeech.csv',
    '/kaggle/input/roman-text-data/collected_data/RomanUrduSentiment.csv',
    '/kaggle/input/roman-text-data/collected_data/Roman_Data_final.csv',
    '/kaggle/input/roman-text-data/collected_data/Roman_Translation_Data.csv',
    '/kaggle/input/roman-text-data/collected_data/urdu roman.txt'
]
file_path = '/kaggle/input/roman-text-data/collected_data/urdu roman.txt'

even_rows = []
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

for i in range(0, len(lines), 2):
    even_rows.append(lines[i].strip())

df_even_rows = pd.DataFrame(even_rows, columns=['Roman_Text'])

def read_file(file_name):
    if file_name.endswith('.csv'):
        return pd.read_csv(file_name)
    elif file_name.endswith('.xlsx'):
        return pd.read_excel(file_name)

dataframes = [read_file(file) for file in colected_files]
dataframes.append(df_even_rows)
collected_data = pd.concat(dataframes, ignore_index=True)

# Remove duplicates
collected_data.drop_duplicates(inplace=True)

# Optionally, reset the index after dropping duplicates
collected_data.reset_index(drop=True, inplace=True)
collected_data

,Roman_Text
0,mil gya hai acha hai aur toota howa b nai hai
1,aram sy pocket may aa jta hai aur quality b ac...
2,sb sy achi chez apki customer service bhot ach...
3,back cover acha hai sai cuts hai aur soft b ha...
4,shru may may thumb grip k khalif thi kyu k ext...
...,...
235622,sabse pehli baat to yeh ke bhai aapne acha kia...
235623,koi aisa jo aap may say kisi nay azmaya hua ho...
235624,acha aap ke jawab se aiik confirmation ho gai
235625,MashAllah ye hui na creativity. zabardast


In [16]:
# collected_data.to_csv('Collected_Dataset.csv', index=False)

## Cleaning 

In [17]:
collected_data = pd.read_csv('/kaggle/input/roman-text-data/Collected_Dataset.csv')
df_stopwords = pd.read_csv('/kaggle/input/roman-text-data/RomanUrdu_stopwords.csv')
collected_data

,Roman_Text
0,sahi bt h
1,"Kya bt hai,"
2,Wah je wah
3,Are wha kaya bat hai
4,Wah kya baat likhi
...,...
235621,sabse pehli baat to yeh ke bhai aapne acha kia...
235622,koi aisa jo aap may say kisi nay azmaya hua ho...
235623,acha aap ke jawab se aiik confirmation ho gai\n
235624,MashAllah ye hui na creativity. zabardast\n


In [18]:
stopwords = set(df_stopwords['you'])

In [19]:
def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub(r"\@w+|\#",'',text)
    text = re.sub(r"[^\w\s]",'',text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    tweet_tokens = word_tokenize(text)
    filtered_tweets=[w for w in tweet_tokens if not w in stopwords] #removing stopwords
    return " ".join(tweet_tokens)

In [ ]:
collected_data['Roman_Text'] = collected_data['Roman_Text'].apply(clean)

In [ ]:
def detect_language(text):
    try:
        # First use langdetect to check if it's English
        lang = detect(text)
        if lang == 'en':
            return 'English'
        
        return 'urdu'  # If it doesn't match any known pattern
    except Exception as e:
        return str(e)

# Apply the language detection function to the Roman_Text column
collected_data['Detected_Language'] = collected_data['Roman_Text'].apply(detect_language)

In [ ]:
collected_data_urdu = collected_data[collected_data['Detected_Language'] != 'English']
collected_data_urdu = collected_data_urdu.reset_index(drop=True)
collected_data_urdu

In [ ]:
collected_data_urdu = collected_data_urdu.drop(['Detected_Language'], axis=1)
# remove less than 5 word sentences 
def filter_short_sentences(text):
    return len(text.split()) >= 5

# Apply the filter function to the DataFrame
collected_data_urdu = collected_data_urdu[collected_data_urdu['Roman_Text'].apply(filter_short_sentences)]
collected_data_urdu = collected_data_urdu.reset_index(drop=True)

In [20]:
# Save the English rows to a new CSV file
# collected_data_urdu.to_csv('Cleaned_Data_Urdu.csv', index=False)

## Translation to make Data

In [ ]:
# Determine the number of rows in each part
num_parts = 6
rows_per_part = len(collected_data_urdu) // num_parts

# Split the DataFrame into 8 equal parts
parts = [collected_data_urdu.iloc[i * rows_per_part:(i + 1) * rows_per_part] for i in range(num_parts)]

# Handle any remaining rows (if the total number of rows is not perfectly divisible by num_parts)
if len(collected_data_urdu) % num_parts != 0:
    parts[-1] = pd.concat([parts[-1], collected_data_urdu.iloc[num_parts * rows_per_part:]])

In [ ]:
for i, part in enumerate(parts):
    part = part.reset_index(drop=True)
    print(len(part))

In [ ]:
# # Save each part to a separate CSV file
# for i, part in enumerate(parts):
#     part.to_csv(f'part_{i + 1}.csv', index=False)

In [ ]:
part_1 = pd.read_csv('/kaggle/input/roman-text-data/Subsets/part_1.csv')

In [ ]:
# Function to translate text with retry mechanism
def translate_with_retry(text, max_retries=3):
    for i in range(max_retries):
        try:
            # Introduce a slight delay to prevent potential rate limiting
            time.sleep(0.5)
            
            # Translate text to English
            translation = GoogleTranslator(source='auto', target='en').translate(text)
            return translation
        except Exception as e:
            print(f"An error occurred during translation: {e}")
            print(f"Retrying translation... (Attempt {i+1}/{max_retries})")
            time.sleep(2)  # Wait for a short duration before retrying
    return "Translation failed after multiple attempts."

# Translate and display one row at a time
for index, row in part_1.iterrows():
    roman_text = row['Roman_Text']
    translation = translate_with_retry(roman_text)
    
    # Add translation to new column 'English_Text'
    part_1.at[index, 'English_Text'] = translation
    
    # Print every 100th row
    if index % 100 == 0:
        print(f'Index: {index}')
        print(f'Roman Urdu: {roman_text}')
        print(f'English Translation: {translation}')
        print()

## Cleaning after Translation

In [ ]:
def clean_data (text): 
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub(r"\@w+|\#",'',text)
    text = re.sub(r"[^\w\s]",'',text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text) #removing stopwords
    return text

In [ ]:
part_1['English_Text'] = part_1['English_Text'].apply(clean_data)

In [ ]:
def detect_language(text):
    try:
        # First use langdetect to check if it's English
        lang = detect(text)
        if lang == 'en':
            return 'English'
        
        return 'urdu'  # If it doesn't match any known pattern
    except Exception as e:
        return str(e)

# Apply the language detection function to the Roman_Text column
part_1['Detected_Language'] = part_1['English_Text'].apply(detect_language)

In [ ]:
part_1_new = part_1[part_1['Detected_Language'] != 'urdu']
part_1_new = translate_5_new.reset_index(drop=True)
part_1_new

In [ ]:
part_1_new = part_1_new.drop(['Detected_Language'], axis=1)
part_1_new

In [ ]:
# part_1_new.to_csv('Final_translation_1.csv', index=False)

## Merging Translated_subsets to make Final Data

In [21]:
file_names = [
    '/kaggle/input/roman-text-data/Translated_Subsets/Final_translation_1.csv',
    '/kaggle/input/roman-text-data/Translated_Subsets/Final_translation_2.csv',
    '/kaggle/input/roman-text-data/Translated_Subsets/Final_translation_3.csv',
    '/kaggle/input/roman-text-data/Translated_Subsets/Final_translation_4.csv',
    '/kaggle/input/roman-text-data/Translated_Subsets/Final_translation_5.csv',
    '/kaggle/input/roman-text-data/Translated_Subsets/Final_translation_6.csv'
]

# Read each CSV file into a DataFrame and store in a list
dataframes = [pd.read_csv(file) for file in file_names]

# Concatenate all DataFrames into one
merged_df = pd.concat(dataframes, ignore_index=True)

# Remove duplicates
merged_df.drop_duplicates(inplace=True)

# Optionally, reset the index after dropping duplicates
merged_df.reset_index(drop=True, inplace=True)

# Save the merged DataFrame to a new CSV file
#merged_df.to_csv('merged_final_translation.csv', index=False)

# Display the first few rows of the merged DataFram
merged_df

,Roman_Text,English_Text
0,are wha kaya bat hai,what are you talking about
1,please everyone allah swt ka naam hamesha bary...,please everyone write the name of allah swt al...
2,ya mere rab tu bra kreem hy is mulk k halat pr...,ya my rab tu bra kreem hy is the mercy on the ...
3,afsos hota hai garip insan kha jay allah ki la...,it is a pity that a strange human being eats a...
4,allah musalman phir dor e jahalat me jaraha h ...,allah has once again shown the muslims the pai...
...,...,...
95551,dost samjhay kahin aap ne mere mutalik kaha,friend understand why you did not say anything...
95552,ab muhjay aj market jana hia chacha k sath aur...,now i have to go to the market with my uncle a...
95553,sabse pehli baat to yeh ke bhai aapne acha kia...,the first thing you should do to this brother ...
95554,koi aisa jo aap may say kisi nay azmaya hua ho...,is there anyone who you might say has tried so...


In [ ]:
# merged_df.to_csv('Finalized_Data_for_Translation.csv', index=False)

In [12]:
df = pd.read_csv('/kaggle/input/roman-text-data/Finalized_Data_for_Translation.csv')
df

,Roman_Text,English_Text
0,are wha kaya bat hai,what are you talking about
1,please everyone allah swt ka naam hamesha bary...,please everyone write the name of allah swt al...
2,ya mere rab tu bra kreem hy is mulk k halat pr...,ya my rab tu bra kreem hy is the mercy on the ...
3,afsos hota hai garip insan kha jay allah ki la...,it is a pity that a strange human being eats a...
4,allah musalman phir dor e jahalat me jaraha h ...,allah has once again shown the muslims the pai...
...,...,...
95551,dost samjhay kahin aap ne mere mutalik kaha,friend understand why you did not say anything...
95552,ab muhjay aj market jana hia chacha k sath aur...,now i have to go to the market with my uncle a...
95553,sabse pehli baat to yeh ke bhai aapne acha kia...,the first thing you should do to this brother ...
95554,koi aisa jo aap may say kisi nay azmaya hua ho...,is there anyone who you might say has tried so...
